In [2]:
from pathlib import Path
import os

import pytesseract
from PIL import Image, ImageDraw

REPO_DIR = Path(os.path.realpath("")).parent
print(REPO_DIR)

/home/jeremy/projects/side_projects/invoice-reader


In [ ]:
image = Image.open(REPO_DIR / ".private/invoice.png").convert("RGB")
image.resize((350,450))

In [ ]:
text = pytesseract.image_to_string(image)
print(text)

In [5]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

In [ ]:
# For each character
image = Image.open(REPO_DIR / ".private/invoice.png").convert("RGB")
boxes = pytesseract.image_to_boxes(image)
print(boxes[:100])
draw = ImageDraw.Draw(image)
width, height = image.size
for box in boxes.splitlines():
    b = box.split(" ")
    bbox = (int(b[1]), height - int(b[4]), int(b[3]), height - int(b[2]))
    # normalized_bbox = normalize_bbox(bbox, width, height)
    draw.rectangle(bbox, outline="red")
image.resize((350,450))

Notes:
* PIL's coordinate system has (0, 0) at the top-left corner, while Tesseract's bounding box coordinates have (0, 0) at the bottom-left corner.
* Rectangle takes x0y0, x1y1 from bottom-left to top-right
* If rectangles are drawn, next OCR will be altered, so it is required reload the image everytime

In [ ]:
# For each word
image = Image.open(REPO_DIR / ".private/invoice.png").convert("RGB")
draw = ImageDraw.Draw(image)
words_dict = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
print(words_dict.keys())
words = [{k: v[i] for k, v in words_dict.items()} for i, _ in enumerate(words_dict["text"])]
for word in words:
    if word["text"].strip() != '':
        x, y, w, h = word['left'], word['top'], word['width'], word['height']
        draw.rectangle([x, y, x + w, y + h], outline="red")
image

In [ ]:
image = Image.open(REPO_DIR / ".private/invoice.png").convert("RGB")
pytesseract.image_to_data(image, output_type=pytesseract.Output.DATAFRAME)